In [1]:
import facebook
import urllib3
import requests
from openpyxl import Workbook
from openpyxl.utils.exceptions import IllegalCharacterError
from datetime import datetime, timezone
import urllib.request
import time

token = "EAACXp46VJZBgBACPLif9vOQhTU6u0sdMDcqbiHhn1jMJVOiArKZBDISyoHk6vOCKkEAKqGsCZBrwOrFTwLelUXBNjAQfudSw5IZBZAzZBgEdC6aukoUxPKygaX8VQpZCYhcRe5zdvXXKQzPbRn02njcGxGkEOg9kEkxM487MKF6VQZDZD"
post_id = "1049379735171791_1578479555595137"
graph = facebook.GraphAPI(access_token=token, version=3.0)

In [2]:
#post = graph.get_object(id='100009925003680_1578479555595137')

"""page = graph.get_object('1049379735171791')
posts = graph.get_connections('1049379735171791','posts')
print(page)
postList = posts['data']
print(postList[4])"""

post = graph.get_object("1049379735171791_1578479555595137")
comments = graph.get_connections("1049379735171791_1578479555595137", 'comments',limit=2000)

"""
Use page_id + '_' + post_id to find post,
then, with the composite id, use get_connections(composite_id, 'comments', limit=n) to get all the comments
n = max amount of comments queried.
"""



print(comments.keys())
print(len(comments['data']))

"""
comments['data'] will return a list of comments
"""


comment_id = comments['data'][302]['id']
print(comment_id)
test_comment = graph.get_object(comment_id,fields='message,attachment,created_time')
print()
print(test_comment['message'])
print()
if 'attachment' in test_comment.keys():
    print(test_comment['attachment']['media']['image']['src'])
else:
    print("No attachment")
print()
print(test_comment['created_time'])

"""
Get the comment ids from the list of comments and query them one by one.
Use get_object with comment_id and fields "message" (for text) and "attachment" (for pictures)
Attachment includes field "src". Use "src" to get image source.
"""

dict_keys(['data', 'paging'])
1056
1578479555595137_1582014508574975

1)ID451044531
2) Nubbie
3) Tiny Warhammer
4) Palladin
5) I think it fits the palladin to have a hammer.
I see him as a dwarf.
6) Works like any hammer with a small radius and high rate of slamming.
Low crit chance.

https://scontent.xx.fbcdn.net/v/t1.0-9/38201147_1049184975259101_8965713590103310336_n.jpg?_nc_cat=0&oh=4c27499a1cd8e6f5a2c813d0832615a0&oe=5C12C001

2018-08-01T18:29:17+0000


'\nGet the comment ids from the list of comments and query them one by one.\nUse get_object with comment_id and fields "message" (for text) and "attachment" (for pictures)\nAttachment includes field "src". Use "src" to get image source.\n'

In [2]:
class FacebookScraper:
    def __init__(self, token, post_id):
        self.graph = facebook.GraphAPI(access_token=token, version=3.0)
        self.post_id = post_id
        self.deadline = datetime(2018, 8, 13, 15, 59, 59, tzinfo=timezone.utc)
        self.wb = Workbook()
        self.ws = self.wb.active
        self.ws.title = "Facebook"
        self.next_row = 1
        self.row_dict = {1: 1}

    def get_comments(self):
        post = self.graph.get_object(self.post_id)
        comments = self.graph.get_connections(self.post_id, 'comments', limit=2000)
        print("Grabbing %d comments" % len(comments['data']))
        return comments  # returns dict of comments

    def parse_comment(self, comment_id):
        comment = self.graph.get_object(comment_id, fields='message,attachment,created_time')
        # comment_id = comment_id
        time.sleep(2)
        comment_text = comment['message']
        if 'attachment' in comment.keys():
            comment_img_src = comment['attachment']['media']['image']['src']
        else:
            comment_img_src = None
        comment_time = comment['created_time']

        return comment_id, comment_text, comment_time, comment_img_src

    def insert_to_worksheet(self):
        comments = self.get_comments()
        for i in range(len(comments['data'])):
            parsed = self.parse_comment(comments['data'][i]['id'])
            if len(parsed[1]) < 30 or datetime.strptime(parsed[2], "%Y-%m-%dT%H:%M:%S%z") > self.deadline:
                continue
            text_data, img_data = list(parsed[:-1]), parsed[-1]
            for i in range(3):
                try:
                    self.ws.cell(row=self.next_row, column=(i + 1), value=text_data[i])
                except IllegalCharacterError:
                    pass
            col = 4
            if img_data:
                # save image
                img_url = img_data
                urllib.request.urlretrieve(img_url,
                                           "/Users/yuhongc/PycharmProjects/SK_Weapon_Event/Facebook_Pictures/%s-%d.jpg" % (
                                               parsed[0], col - 3))
                # insert to worksheet

                col += 1
            self.next_row += 1
            if self.next_row % 100 == 0:
                print("Processed %d rows" % self.next_row)
                self.save(self.next_row)

    def save(self, checkpoint=None):
        if not checkpoint:
            self.wb.save('Facebook.xlsx')
        else:
            self.wb.save('Facebook-checkpoint-%d.xlsx' % checkpoint)


In [3]:
FBScraper = FacebookScraper(token, post_id)

In [4]:
FBScraper.insert_to_worksheet()

GraphAPIError: (#4) Application request limit reached